In [1]:
import numpy as np
import cv2
import skimage

In [2]:
# Generate test files
h, w = 800, 800
marker_size = 100

# Generate backdrop
backdrop = np.zeros((h, w, 3), dtype=np.uint8)
backdrop += 255
marker_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
my_marker = cv2.aruco.generateImageMarker(marker_dict, 0, marker_size)
my_marker = np.expand_dims(my_marker, 2)
my_marker[:, :, 1:] = 0
backdrop[:my_marker.shape[0], :my_marker.shape[1]] = my_marker
backdrop[:my_marker.shape[0], :my_marker.shape[1], 0] = 255

In [18]:
def rectify_scan(filename):
    baseline_scan = cv2.imread(filename)
    detector = cv2.aruco.ArucoDetector(marker_dict, cv2.aruco.DetectorParameters())
    corners, ids, rejected = detector.detectMarkers(baseline_scan)
    corners = np.array(corners[0][0])
    expected_marker_corners = np.array([
        [0,0],
        [marker_size, 0],
        [marker_size, marker_size],
        [0, marker_size]
    ])
    H = cv2.getPerspectiveTransform(corners.astype(np.float32), expected_marker_corners.astype(np.float32))
    baseline_scan = cv2.warpPerspective(baseline_scan, H, (h, w))
    return baseline_scan

In [30]:
lower_red1 = np.array([0, 30, 50])
upper_red1 = np.array([20, 255, 255])

lower_red2 = np.array([160, 30, 50])
upper_red2 = np.array([180, 255 , 255])

red_range = [(lower_red1, upper_red1), (lower_red2, upper_red2)]

lower_black = np.array([0, 0, 0])
upper_black = np.array([180, 255, 90])

black_range = (lower_black, upper_black)

def get_mask(bgr_img: np.ndarray, ranges: tuple[np.ndarray] | list[tuple[np.ndarray]]) -> np.ndarray[np.bool]:
    hsv = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2HSV)
    if(type(ranges) is tuple):
        ranges = [ranges]

    mask = None
    for range in ranges:
        lower = range[0]
        upper = range[1]
        if mask is None:
            mask = cv2.inRange(hsv, lower, upper)
        else:
            mask = mask | cv2.inRange(hsv, lower, upper)

    return mask


In [19]:
# Generate circle
# circle = backdrop.copy()
# cv2.circle(circle, (w//2 * 1, h//2), 200, (0, 0, 255), thickness=1, lineType=cv2.LINE_AA)
# cv2.imwrite("circle.png", circle)

# Extract baseline skeleton

mask_red = get_mask(rectify_scan("circle_scan.jpg"), red_range)
mask_uint8 = mask_red.astype('uint8')
cv2.imshow("mask", mask_uint8)
cv2.waitKey(0)
circle_skeleton = skimage.morphology.skeletonize(mask_red > 0)
skeleton_uint8 = (circle_skeleton.astype('uint8')) * 255
cv2.imshow("skeleton", skeleton_uint8)
cv2.waitKey(0)

-1

In [6]:
circle_filled = backdrop.copy()
cv2.circle(circle_filled, (w//2 * 1, h//2), 200, (0, 0, 255), thickness=-1, lineType=cv2.LINE_AA)
cv2.imwrite("circle_filled.png", circle_filled)

True

In [34]:
# Read drawing
circle_drawn = rectify_scan("circle_scan_drawn.jpg")
mask_drawn = get_mask(circle_drawn, black_range)
mask_uint8 = mask_drawn.astype('uint8')
cv2.imshow("mask", mask_uint8)
cv2.waitKey(0)
circle_drawn_skeleton = skimage.morphology.skeletonize(mask_drawn > 0)
skeleton_uint8 = (circle_drawn_skeleton.astype('uint8')) * 255
cv2.imshow("skeleton", skeleton_uint8)
cv2.waitKey(0)


-1

In [32]:
# Computer per-pixel distance of every pixel against the reference
distance_drawn_to_gt = cv2.distanceTransform((~circle_skeleton).astype(np.uint8), cv2.DIST_L2, 3)
distance_drawn_to_gt[~circle_drawn_skeleton] = 0
distance_gt_to_drawn = cv2.distanceTransform((~circle_drawn_skeleton).astype(np.uint8), cv2.DIST_L2, 3)
distance_gt_to_drawn[~circle_skeleton] = 0

error_d_gt = np.sum(distance_drawn_to_gt)
count_d_gt = np.count_nonzero(distance_drawn_to_gt)
error_gt_d = np.sum(distance_gt_to_drawn)
count_gt_d = np.count_nonzero(distance_gt_to_drawn)
asymmetric = error_d_gt/count_d_gt
symmetric = (error_d_gt/count_d_gt + error_gt_d/count_gt_d) / 2

print(f"Asymmetric error (Deviation from path): {asymmetric:3f}")
print(f"Symmetric error (Completeness): {symmetric:3f}")
distance_drawn_to_gt = distance_drawn_to_gt.astype(np.float32)
# distance_drawn_to_gt /= np.max(distance_drawn_to_gt)
cv2.imshow("distance", distance_drawn_to_gt)
cv2.waitKey(0)

Asymmetric error (Deviation from path): 4.468151
Symmetric error (Completeness): 4.087247


-1

In [9]:
# Read filled drawing
circle_drawn = cv2.imread("circle_filled_drawn.png")
mask_drawn = get_mask(circle_drawn, black_range)
mask_gt = get_mask(circle_filled, red_range)
filled_correct = mask_drawn & mask_gt
filled_incorrect = mask_drawn & ~mask_gt
total_gt = np.count_nonzero(mask_gt)
print(f"Correctly filled %: {np.count_nonzero(filled_correct)/total_gt}")
print(f"Incorrectly filled %: {np.count_nonzero(filled_incorrect)/total_gt}")

Correctly filled %: 1.0
Incorrectly filled %: 0.25438963045894997


In [38]:
error_nonzero = distance_drawn_to_gt[distance_drawn_to_gt != 0].flatten()
print("Mean error: ", np.mean(error_nonzero))
print("95th percentile error: ", np.percentile(error_nonzero, 95))
print("Stddev: ", np.std(error_nonzero))
error_lt_threshold = error_nonzero[error_nonzero < 1]
print("Below threshold: ", error_lt_threshold.shape[0] / error_nonzero.shape[0])

Mean error:  4.4681506
95th percentile error:  9.9643
Stddev:  9.64898
Below threshold:  0.1345475910693302
